In [ ]:
# import the necessary packages
import os
from skimage import measure
import numpy as np
import argparse
import cv2
from support import show_images_gray
import support
import contours
from scipy.spatial import distance as dist
from centroidtracker import CentroidTracker
import time
import glob
import scipy.misc
from PIL import Image

def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

#path to input image
imgdir = "Insert/Base/Directory/Here/"
basename = '*.jpg'

#width of leftmost object in scene.  Turn off with False
leftmostwidth = False #250

#size of square in calibration grid in pixels only turn on one
#squarepix = 120 #for 202mm
#squarepix = 70  #for 302mm
squarepix = 165  #for Echo 176

#size of square in calibration grid in microns
squaremicron = 250


#Blur?
blur = True

#blur amount.  Gaussian.  use odd, equal numbers.  0,0 computes from sigmoid
blurx = 5
blury = 5

#erode and dilate?
erosion = True

#erosion and dilation iterations
erode_iter = 3
dilate_iter = 3

#blob size filtering. minimum pixels and max pixels per blob. +/-10% suggested.
minpix = USERDEFINED
maxpix = USERDEFINED


In [ ]:
%%time
#Load in images.  Best if image sequence, but can run ffmpeg if needed
print("Loading images to memory...")

filenames = glob.glob(imgdir+basename)
print(len(filenames))


In [ ]:
%%time
#crop wells to dogbone rectangles


if not os.path.exists(imgdir+"Crop/"):
    os.mkdir(imgdir+"Crop/")
    print("Directory " , (imgdir+"Crop/") ,  " Created ")
else:
    print("Directory " , (imgdir+"Crop/") ,  " already exists")


for (i,files) in enumerate(filenames):
    image = Image.open(files)
    imgwidth, imgheight = image.size
    imheight = imgheight//3
    imwidth = imgwidth//4
    temp = image.crop((imwidth-300, imheight-100, (3*imwidth+400), (2*imheight-50)))
    filename = imgdir + 'Crop/CropWell' + str(files[-7:-4]).zfill(4) + '.tif'
    temp.save(filename)
    print('█', end = '') 
    
filenames = glob.glob(imgdir+"Crop/"+"*.tif")



In [ ]:
%%time
print("Extracting features... This might take a while...")
t_start = time.time()

images = []


# load the image, convert it to grayscale, and blur it
for files in filenames: images.append(cv2.imread(files))

images = np.asarray(images)
total_images = images.shape[0]
grays, blurreds, threshes, dilateds = [],[],[],[]

for (i,image) in enumerate(images):
    temp = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grays.append(temp)

grays = np.asarray(grays)
print("... Grayscale Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Gray length: ", grays.shape)
    
for (i,gray) in enumerate(grays):
    temp = gray
    if blur:
        temp = cv2.GaussianBlur(gray, (blurx, blury), 0)
    blurreds.append(temp)

blurreds = np.asarray(blurreds)    
print("... Blurring Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Blurred length: ", blurreds.shape) 

# Create target Directory if does not exist
if not os.path.exists(imgdir+"Thresh/"):
    os.mkdir(imgdir+"Thresh/")
    print("Directory " , (imgdir+"Thresh/") ,  " Created ")
else:
    print("Directory " , (imgdir+"Thresh/") ,  " already exists")

for (i,blurred) in enumerate(blurreds):
    temp = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, USERDEFINED, 2)#[1]
    threshes.append(temp)
    wellnumber = i
    filename = imgdir + 'Thresh/ThrWell' + str(wellnumber).zfill(4) + '.jpg'
    Image.fromarray(temp).save(filename)
    print('█', end = '') 
    
threshes = np.asarray(threshes)
print("... Thresholding Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Threshes length: ", threshes.shape)  

# Create target Directory if does not exist

if not os.path.exists(imgdir+"Dilated/"):
    os.mkdir(imgdir+"Dilated/")
    print("Directory " , (imgdir+"Dilated/") ,  " Created ")
else:
    print("Directory " , (imgdir+"Dilated/") ,  " already exists")

for (i,thresh) in enumerate(threshes):
    temp = thresh
    if erosion:
        temp = cv2.erode(thresh, None, iterations=erode_iter)
        temp = cv2.dilate(temp, None, iterations=dilate_iter)
    dilateds.append(temp)
    wellnumber = i + 1
    filename = imgdir + 'Dilated/DilWell' + str(wellnumber) + '.jpg'
    Image.fromarray(temp).save(filename)
    print('█', end = '') 
    
dilateds = np.asarray(dilateds)
print("... Eroding and Dilating Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Dilateds length: ", dilateds.shape) 



In [ ]:
%%time
# perform a connected component analysis on the DILATED
# image, then initialize a mask to store only the 
# components of interest

#create masks directory if it does not exist
if not os.path.exists(imgdir+"Mask/"):
    os.mkdir(imgdir+"Mask/")
    print("Directory " , (imgdir+"Mask/") ,  " Created ")
else:
    print("Directory " , (imgdir+"Mask/") ,  " already exists")
    
masks = []
for (i,dilated) in enumerate(dilateds):
    labels = measure.label(dilated, neighbors=8, background=0)
    mask = np.zeros(dilated.shape, dtype="uint8")
    
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(dilated.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        
        #if aspect ratio of bounding rectangle is not close to square, cont
#         rect = cv2.minAreaRect(labelMask)
#         print(rect)
        
        # if the number of pixels in the component falls within a range,
        # then add it to our mask list
        if minpix < numPixels < maxpix:
            mask = cv2.add(mask, labelMask)
    
           
    masks.append(mask)
    wellnumber = i
    filename = imgdir + 'Mask/MskWell' + str(wellnumber).zfill(4) + '.jpg'
    Image.fromarray(mask).save(filename)
    print('█', end = '') 
        
masks = np.asarray(masks)    
print("... Discrete Components Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Masks length: ", np.shape(masks)) 

In [ ]:
%%time
for (i,x) in enumerate(masks):
    show_images_gray([grays[i],blurreds[i],threshes[i],masks[i]], per_row = 4, per_col = 1, W = 10, H = 2)

In [ ]:
%%time
# find the contours in the mask, then sort them from left to
# right

distances = []
boxed = []

#create Contours directory if it does not exist
if not os.path.exists(imgdir+"Contour/"):
    os.mkdir(imgdir+"Contour/")
    print("Directory " , (imgdir+"Contour/") ,  " Created ")
else:
    print("Directory " , (imgdir+"Contour/") ,  " already exists")

for(i,mask) in enumerate(masks):
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    
    cnts = convenience.grab_contours(cnts)
    #add a contour if none is present to prevent crash on blank well.  Will not show in final measurements
    if not cnts:
        cnts = [np.array([[1,1],[5,5],[10,10]], dtype=np.int32)]
    cnts = contours.sort_contours(cnts)[0]
 
    color = (0,0,255)
    refObj = None
    
    # loop over the contours individually
    for c in cnts:
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 3:
            continue
    
        # compute the rotated bounding box of the contour
        box = cv2.minAreaRect(c)
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
    
        # order the points in the contour such that they appear
        # in top-left, top-right, bottom-right, and bottom-left
        # order, then draw the outline of the rotated bounding
        # box
        box = support.order_points(box)
        
    
        # compute the center of the bounding box
        cX = np.average(box[:, 0])
        cY = np.average(box[:, 1])
        
        #calculate aspect ratio of bounding box and reject rectangles
        (tl, tr, br, bl) = box
        topwidth = dist.euclidean(tl,tr)
        leftheight = dist.euclidean(tl,bl)
        biglength = max(topwidth,leftheight)
        minlength = min(topwidth,leftheight)
        aspectratio = biglength / minlength
        print(topwidth, leftheight, aspectratio)
        if (aspectratio > 1.2):
            print("Skipped Object")
            continue       
        

        # if this is the first contour we are examining (i.e.,
        # the left-most contour), we presume this is the
        # reference object
        if refObj is None:
            # unpack the ordered bounding box, then compute the
            # midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-right and
            # bottom-right
            (tl, tr, br, bl) = box
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)
 
            # compute the Euclidean distance between the midpoints,
            # then construct the reference object
            D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
            if leftmostwidth:
                refObj = (box,(cX, cY), D / leftmostwidth) #D/leftmostwidth calculates width from object size. fibers too bright
            else:
                refObj = (box,(cX, cY), squarepix / squaremicron) #include this line to bypass width det by object size
            #print(refObj)
            continue
    
        # draw the contours on the image
        orig = images[i].copy()
        cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 10)
        cv2.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 10)
    
        # stack the reference coordinates and the object coordinates
        # to include the object center
        refCoords = np.vstack([refObj[0], refObj[1]])
        objCoords = np.vstack([(cX, cY)])
        
        (xA,yA) = refCoords[4]
        
        xB = cX
        yB = cY
            # draw circles corresponding to the current points and
            # connect them with a line
        cv2.circle(orig, (int(xA), int(yA)), 1, (255,0,0), -1)
        cv2.circle(orig, (int(xB), int(yB)), 1, (255,0,0), -1)
        cv2.line(orig, (int(xA), int(yA)), (int(xB), int(yB)),(255,255,255),10) 
        
        # compute the Euclidean distance between the coordinates,
        # and then convert the distance in pixels to distance in
        # units
        D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]
        (mX, mY) = midpoint((xA, yA), (xB, yB))
        cv2.putText(orig, "{:.1f}um".format(D), (int(mX-550), int(mY - 150)),cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255,255), 10)
        distances.append(D)
        wellnumber = i
        filename = imgdir + 'Contour/CntWell' + str(wellnumber).zfill(4) + '.jpg'
        Image.fromarray(orig).save(filename)
        print('█', end = '')
        
    
    
              
    

In [ ]:
textfilename = imgdir + "raw_distances.csv"
np.savetxt(textfilename, distances, delimiter=',', fmt='%f')

In [ ]:
print(D)


In [ ]:
print(distances)